## Import our favorite friends

In [ ]:
import numpy as np #numpy will read the image into an array for source extraction 
import sep #sep is a source exraction and photometry library 

## Import FITS, matplotlib, set figure parameters

In [ ]:
from astropy.io import fits #flexible image transport system (fits) is a portable file standard used to store images and tables
import matplotlib.pyplot as plt #needed to show resultant images
from matplotlib import rcParams #rcParams allows us to set color bar size, color, shape, origin, axes.

#the plots will be displayed below the matplotlib commands and will be stored in the notebook
%matplotlib inline 

rcParams['figure.figsize'] = [10., 8.] #set the default size of figures for the notebook 

## Read the image into a 2-D array 

In [ ]:
fname = "original image.fits" #rename the tutorial image 
hdu_list = fits.open(fname) #opens the header data unit (highest level component of fits; includes a header and a data array or table)
hdu_list.info() #summarizes content of the opened fits file

## Get dimensions of image

In [ ]:
image_data = fits.getdata(fname) #gets data from the image 
print(type(image_data)) #prints the type of data 
print(image_data.shape) #prints the data shape (dimensions in (x,y)).

## Show image data

In [ ]:
plt.imshow(image_data, cmap='gray', origin = 'lower') #shows the image constructed from the array data using a gray color map.
plt.colorbar() #shows the color bar on the right hand side of the image.

## Get additional data for fun

In [ ]:
print('Min:', np.min(image_data)) #prints the minimum value in the data array 
print('Max:', np.max(image_data)) #prints the maximum value in the data array 
print('Mean:', np.mean(image_data)) #prints the mean value in the data array 
print('Stdev:', np.std(image_data)) #prints the standard deviation of the values in the data array 

## Get a better quality image using LogNorm

In [ ]:
from matplotlib.colors import LogNorm #lognorm normalizes the given value to a range of 0 to 1 on a logarithmic scale

In [ ]:
plt.imshow(image_data, cmap='gray', norm=LogNorm(), origin = 'lower') #shows image with gray color map, normalize on a log scale so the bright features show up better 

cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4]) #color bar on the right hand side should have ticks set at 5000, 10000, and 20000
cbar.ax.set_yticklabels(['5,000', '10,000','20,000']) #set the tick labels on the y-axes color bar at 5k, 10k, and 20k.

plt.savefig('fig1.png',bbox_inches='tight', dpi=300) #save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.

In [ ]:
bkg = sep.Background(image_data) #create an array of the background using default settings

In [ ]:
print(bkg.globalback) #prints the global mean of the image background
print(bkg.globalrms) #prints the global noise of the image background

In [ ]:
bkg_image = bkg.back() #set a 2-d array of background data, same size as the original image

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
#interpolation = 'nearest' means pixels are shown as a square of multiple pixels - works well when small images are scaled up.
#cmap = 'gray' uses gray color map. Could use magma, plasma, inferno, etc..
#origin ='lower' sets the origin to the lower left hand corner
plt.colorbar(); #displays the color bar
plt.savefig('fig2.png',bbox_inches='tight', dpi=300) #save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.

In [ ]:
bkg_rms = bkg.rms() #set the background error (root mean square error)

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower') 
#interpolation = 'nearest' means pixels are shown as a square of multiple pixels - works well when small images are scaled up.
#cmap = 'gray' uses gray color map. Could use magma, plasma, inferno, etc..
#origin ='lower' sets the origin to the lower left hand corner
plt.colorbar(); #displays the color bar
plt.savefig('fig3.png',bbox_inches='tight', dpi=300) #save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.

In [ ]:
data_sub = image_data - bkg #subtracts the background from the image data to get new data for analysis

### Detect objects on the background-subtracted data

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms) #extracts the objects from the new data

In [ ]:
number_of_objects = len(objects) #number_of_objects is equal to the length of the object list
print(number_of_objects) #prints the length of the object list
#could have written len(objects) instead 

In [ ]:
# Import Ellipse module for drawing ellipses around the objects in the image
from matplotlib.patches import Ellipse 

#Plot the background-subtracted image
fig, ax = plt.subplots() #allows combination of multiple plots on a single figure with axes.
m, s = np.mean(data_sub), np.std(data_sub) #compute the mean and standard deviation from the data_sub array elements

im = ax.imshow(data_sub, 
               interpolation='nearest', #'nearest' means pixels are shown as a square of multiple pixels - works well when small images are scaled up.
               cmap='gray', #cmap = 'gray' uses gray color map. Could use magma, plasma, inferno, etc..
               vmin=m-s, vmax=m+s, origin='lower') #vmin = m-s will map the color scale linearly so white is vmax and darjer gray is vmin.

#Plot one ellipse around each object
for i in range(len(objects)): #for all i between 0 and number of objects
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i], # Width (semi-major axis) of ellipse is 6 times the object. 
                height=6*objects['b'][i], # Height (semi-minor axis) of ellipse is 6 times the height.
                angle=objects['theta'][i] * 180. / np.pi) # Angle is theta converted to degrees
    e.set_facecolor('none') # Ellipses will NOT be filled in
    e.set_edgecolor('red') # The color of the ellipse is red
    ax.add_artist(e)       # Adds the ellipse subplot to the image subplot for an entire image
    
#Save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.
plt.savefig('fig4.png',bbox_inches='tight', dpi=300) 

### Aperture photometry - Perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects

In [ ]:
# Calculate the flux, flux error
# Flux is the total amount of energy crossing a unit area per unit time. 
# Fluxerr is the flux error; (sigma^2)_F = sigma^2 summed over i + F/g
# Sigma = the noise = the globalrms from the background. 
# Set F = 3.0; sum of the flux within the radius of 3 pixels
# gain is the poisson uncertainty

flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0) 



In [ ]:
# Print the first 10 fluxes and their uncertainty 
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i])) 

### Plot the histogram o the fluxes

In [ ]:
 # Convert the flux data array to 1D
# Plot the Histogram of the fluxes. 
# X-axis - magnitude of the flux
# Y-axis - Number of objects with similar flux
# bins  - Number of rectangles the data is split into. Auto value will choose best fit.

histogram = plt.hist(flux.flatten(), bins='auto')
